### 국회의원 정보 스크래핑
* 국회의원이름, 국회의원id 추출
* 국회의원 상세페이지에 300번 요청을 보내서 상세정보 추출,
* pandas Dataframe에 저장
* 시각화(막대그래프, 히스토그램, 파이차트, )
* DB에 members 테이블에 저장

In [9]:
import requests
from bs4 import BeautifulSoup
import re
url='https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do?currentPage=1&rowPerPage=300'

html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

atag_list = soup.select('.memberna_list dl dt a')
print(len(atag_list))

id_list = list()
for idx,atag in enumerate(atag_list, 1):
    #print(atag)
    name = atag.text
    link = atag['href']
    matched = re.search(r"(\d+)", link)
    member_id = matched.group(0)
    #print(name, member_id)
    id_list.append(member_id)
    #print(name, link)
    
#id_list
    

300


In [47]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

print('시작-----------------------------------------')
member_list = []
for idx, id in enumerate(id_list, 1): # id_list 하면 300개나오니까 잠시 이렇게 ///(id_list[0:10], 1)
    detail_url =f'https://www.assembly.go.kr/assm/memPop/memPopup.do?dept_cd={id}'
    #print(idx, detail_url)
    print(idx, id)
    html = requests.get(detail_url).text
    soup = BeautifulSoup(html, 'html.parser')
    #한명의 상세정보를 저장하는 딕셔너리
    member_dict = {}
    
    ##dt tag만 
    dt_list = []
    for dt_tag in soup.select('.info_mna dl.pro_detail dt'):
        #print(dt_tag)
        #텍스트 뽑아내서 리스트에 추가
        dt_list.append(dt_tag.text) 
    #print(dt_list)   
   
    dd_list = []
    for dd_tag in soup.select('.info_mna dl.pro_detail dd'):
        regex = re.compile(r'[\n\r\t]')
        result = regex.sub('',dd_tag.text.strip().replace(" ",""))
        #result = regex.sub('',dd_tag.text.strip()) 
        # 중간공백 비우기 위해
        #sub("", a) a를 ""으로 치환
        #replace("검색문자","치환문자", "치환횟수")
        #텍스트 뽑아내서 리스트에 추가  
        dd_list.append(result) 
    #print(dd_list)     
    
    
    member_dict = dict(zip(dt_list,dd_list))
    #print(member_dict)    

    for div_tag in soup.select('.info_mna .profile'):
        #print(div_tag)
        member_dict['이름'] = div_tag.select('h4')[0].text #0쓰는 이유 select가 리스트로 반환
        #print(name)  # 조금있다 해보기
        img_url = div_tag.find('img').attrs['src']
        member_dict['이미지'] = urljoin(detail_url,img_url)
        member_dict['생년월일'] = div_tag.select_one("li:nth-of-type(4)").text
        #해당하는 자식 태그의 순서를 정해줌
        #print(img_url)
        #print(full_url)
        #print(birth_date)
        
    #print(member_dict)        
    member_list.append(member_dict)    
print(len(member_list)) 
print('작업종료-------------------------------')    

시작-----------------------------------------
1 9771230
2 9771142
3 9771174
4 9771233
5 9771283
6 9770933
7 9771116
8 9771276
9 9771168
10 9771007
11 9771109
12 9771180
13 9770931
14 9770956
15 9771224
16 9771165
17 9770679
18 9771108
19 9770977
20 9771259
21 9770998
22 9770927
23 9771258
24 9770712
25 9771152
26 9771164
27 9771181
28 9770719
29 9770997
30 9771136
31 9770723
32 9771120
33 9771173
34 9770941
35 9770984
36 9771285
37 9771268
38 9770725
39 9770527
40 9771024
41 9771195
42 9770985
43 9771208
44 9771089
45 9771210
46 9771220
47 9771144
48 9771169
49 9771111
50 9771225
51 9770975
52 9770981
53 9770934
54 9771249
55 9771186
56 9771147
57 9771127
58 9771214
59 9771206
60 9771171
61 9771023
62 9771090
63 9771009
64 9771192
65 9771281
66 9770982
67 9770986
68 9770741
69 9771237
70 9770742
71 9770991
72 9771223
73 9771270
74 9771216
75 9771134
76 9771228
77 9770752
78 9770753
79 9770755
80 9771141
81 9771274
82 9771091
83 9771187
84 9771204
85 9771175
86 9771159
87 9770764
88 97709

In [48]:
member_list[298:]

[{'정당': '더불어민주당',
  '선거구': '대전중구',
  '소속위원회': '산업통상자원중소벤처기업위원회,예산결산특별위원회',
  '당선횟수': '초선(21대)',
  '사무실전화': '02-784-1935',
  '홈페이지': '',
  '이메일': 'unhah0301@gmail.com',
  '보좌관': '박승규,방인호',
  '비서관': '장진섭,최형준',
  '비서': '유혜정,홍승경,이제문,정윤주,정서원',
  '취미, 특기': '',
  '이름': '황운하',
  '이미지': 'https://www.assembly.go.kr/photo/9771161.jpg',
  '생년월일': '1962-09-10'},
 {'정당': '더불어민주당',
  '선거구': '서울양천구갑',
  '소속위원회': '국방위원회',
  '당선횟수': '재선(20대,21대)',
  '사무실전화': '02-784-8551',
  '홈페이지': 'http://blog.naver.com/hwanghee67',
  '이메일': 'hwanghee6728@gmail.com',
  '보좌관': '나성채,이용협',
  '비서관': '김순구,김진하',
  '비서': '이정수,이종서,장유리,강조원,나종호',
  '취미, 특기': '',
  '이름': '황희',
  '이미지': 'https://www.assembly.go.kr/photo/9770936.jpg',
  '생년월일': '1967-07-28'}]

In [55]:
# [{}, {}, {}]
# {} -> series 한개의 row
# [] -> dataframe

import pandas as pd
#데이터프레임 만들기
data_df = pd.DataFrame(columns = ['이름', '이미지', '생년월일','정당','소속위원회','당선횟수','사무실전화','홈페이지','이메일','보좌관'\
                                  ,'비서관','비서','취미, 특기'
                                 ])
#data_df 데이터 프레임 만듦

# 줄 1개가 시리즈  , 사실 얘가 한개의 로우
for member in member_list:
    series_obj = pd.Series(member)
    #print(member)
    data_df = data_df.append(series_obj, ignore_index=True) #인덱스 무시
    
data_df.head()

,이름,이미지,생년월일,정당,소속위원회,당선횟수,사무실전화,홈페이지,이메일,보좌관,비서관,비서,"취미, 특기",선거구
0,강기윤,https://www.assembly.go.kr/photo/9771230.jpg,1960-06-04,미래통합당,보건복지위원회,"재선(19대,21대)",02-784-1751,http://blog.naver.com/ggotop,ggotop@naver.com,"강종길,김홍광","박응서,최광림","김영록,안효상,이유진,홍지형,김지훈",,경남창원시성산구
1,강대식,https://www.assembly.go.kr/photo/9771142.jpg,1959-11-02,미래통합당,국방위원회,초선(21대),,,,"박홍규,정운태",유진영,"박종원,박순권,김광연,김현정,송민욱",,대구동구을
2,강득구,https://www.assembly.go.kr/photo/9771174.jpg,1963-05-27,더불어민주당,교육위원회,초선(21대),02-784-2747~9,https://blog.naver.com/dulipapa,mainsail440@daum.net,유진우,"김도균,홍미하","최경순,김혜원,최기섭,조나연,오세령",,경기안양시만안구
3,강민국,https://www.assembly.go.kr/photo/9771233.jpg,1971-03-03,미래통합당,정무위원회,초선(21대),,,,"강민승,박진우","신우성,오경훈","성환종,김민규,김오주,정재희,이승연",,경남진주시을
4,강민정,https://www.assembly.go.kr/photo/9771283.jpg,1961-04-26,열린민주당,교육위원회,초선(21대),,,,전경원,"김민혜,윤호숙","김수안,김성용,민지홍,서지혜,이태경",,비례대표


In [58]:
print(data_df.columns)
print(data_df.index)
print(data_df.shape)
data_df.info()

Index(['이름', '이미지', '생년월일', '정당', '소속위원회', '당선횟수', '사무실전화', '홈페이지', '이메일',
       '보좌관', '비서관', '비서', '취미, 특기', '선거구'],
      dtype='object')
RangeIndex(start=0, stop=300, step=1)
(300, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이름      300 non-null    object
 1   이미지     300 non-null    object
 2   생년월일    300 non-null    object
 3   정당      300 non-null    object
 4   소속위원회   300 non-null    object
 5   당선횟수    300 non-null    object
 6   사무실전화   300 non-null    object
 7   홈페이지    300 non-null    object
 8   이메일     300 non-null    object
 9   보좌관     300 non-null    object
 10  비서관     300 non-null    object
 11  비서      300 non-null    object
 12  취미, 특기  300 non-null    object
 13  선거구     300 non-null    object
dtypes: object(14)
memory usage: 32.9+ KB


In [60]:
#정당의 유니크한 값
data_df['정당'].unique()

array(['미래통합당', '더불어민주당', '열린민주당', '정의당', '무소속', '국민의당', '기본소득당', '시대전환'],
      dtype=object)

In [62]:
data_df['정당'].value_counts()

더불어민주당    176
미래통합당     103
무소속         7
정의당         6
열린민주당       3
국민의당        3
시대전환        1
기본소득당       1
Name: 정당, dtype: int64

In [64]:
data_df['선거구'].unique()[0:3] # head() 안먹혀 -> array라서

array(['경남창원시성산구', '대구동구을', '경기안양시만안구'], dtype=object)

In [65]:
data_df['당선횟수'].unique()

array(['재선(19대,21대)', '초선(21대)', '재선(20대,21대)', '4선(18대,19대,20대,21대)',
       '4선(16대,17대,18대,21대)', '3선(19대,20대,21대)', '재선(17대,21대)',
       '4선(17대,18대,19대,21대)', '3선(15대,16대,21대)', '4선(17대,19대,20대,21대)',
       '재선(18대,21대)', '5선(17대,18대,19대,20대,21대)', '3선(18대,19대,21대)',
       '6선(16대,17대,18대,19대,20대,21대)', '5선(16대,17대,18대,19대,21대)',
       '5선(15대,16대,19대,20대,21대)', '5선(16대,17대,18대,20대,21대)',
       '3선(17대,19대,21대)', '3선(17대,18대,21대)', '3선(18대,20대,21대)',
       '5선(15대,16대,17대,18대,21대)'], dtype=object)

In [66]:
data_df['소속위원회'].unique()

array(['보건복지위원회', '국방위원회', '교육위원회', '정무위원회', '국회운영위원회,보건복지위원회', '환경노동위원회',
       '국토교통위원회', '산업통상자원중소벤처기업위원회', '기획재정위원회', '국회운영위원회,교육위원회',
       '농림축산식품해양수산위원회', '행정안전위원회', '교육위원회,여성가족위원회', '기획재정위원회,정보위원회',
       '외교통일위원회,예산결산특별위원회', '법제사법위원회', '국회운영위원회,법제사법위원회',
       '보건복지위원회,여성가족위원회', '행정안전위원회,예산결산특별위원회', '국방위원회,정보위원회',
       '과학기술정보방송통신위원회', '외교통일위원회', '국회운영위원회,환경노동위원회', '문화체육관광위원회',
       '국회운영위원회,행정안전위원회', '국회운영위원회,농림축산식품해양수산위원회', '보건복지위원회,예산결산특별위원회',
       '국회운영위원회,산업통상자원중소벤처기업위원회,여성가족위원회', '교육위원회,예산결산특별위원회',
       '국회운영위원회,외교통일위원회,정보위원회', '국회운영위원회,기획재정위원회', '정무위원회,예산결산특별위원회',
       '외교통일위원회,정보위원회', '국회운영위원회,국토교통위원회', '환경노동위원회,정보위원회',
       '국회운영위원회,과학기술정보방송통신위원회', '', '국회운영위원회,국방위원회', '기획재정위원회,예산결산특별위원회',
       '국회운영위원회,정무위원회', '법제사법위원회,예산결산특별위원회', '법제사법위원회,여성가족위원회',
       '국회운영위원회,산업통상자원중소벤처기업위원회', '기획재정위원회,여성가족위원회',
       '산업통상자원중소벤처기업위원회,여성가족위원회', '환경노동위원회,예산결산특별위원회',
       '농림축산식품해양수산위원회,예산결산특별위원회', '문화체육관광위원회,여성가족위원회',
       '과학기술정보방송통신위원회,예산결산특별위원회', '국방위원회

In [71]:
# 당선횟수2 컬럼을 생성

type(data_df['당선횟수']) # series 타입
temp_str = data_df['당선횟수'].str
type(temp_str) #stringmethods 타입 - 스트링에서 제공하는 것들을 쓸 수 있음
# 잠깐 문자열 타입으로 바꿈
numberof_election = temp_str[:2]
data_df['당선횟수2'] = numberof_election
data_df.loc[:,['당선횟수', '당선횟수2']].head()
data_df['당선횟수2'].value_counts()

초선    151
재선     74
3선     42
4선     19
5선     13
6선      1
Name: 당선횟수2, dtype: int64

In [78]:
data_df.loc[data_df['당선횟수2']=='6선']

,이름,이미지,생년월일,정당,소속위원회,당선횟수,사무실전화,홈페이지,이메일,보좌관,비서관,비서,"취미, 특기",선거구,당선횟수2
92,박병석,https://www.assembly.go.kr/photo/9770090.jpg,1952-01-25,무소속,,"6선(16대,17대,18대,19대,20대,21대)",02-788-2306,,pbs2100@gmail.com,"강창희,송석근","예석해,최명식","김혜리,김민숙,유민선,심용진,최지연",,대전서구갑,6선


In [81]:
data_df['생년월일'].head()

0    1960-06-04
1    1959-11-02
2    1963-05-27
3    1971-03-03
4    1961-04-26
Name: 생년월일, dtype: object

In [85]:
# Datetimeindex 클래스를 이용해서 생년월일 컬럼에서 년/월/일 값을 추출
# year, month, day 세개의 컬럼을 추가

data_df['year']=pd.DatetimeIndex(data_df['생년월일']).year
data_df['month']=pd.DatetimeIndex(data_df['생년월일']).month
data_df['day']=pd.DatetimeIndex(data_df['생년월일']).day
data_df.loc[:, ['생년월일','year','month','day']].head()

,생년월일,year,month,day
0,1960-06-04,1960,6,4
1,1959-11-02,1959,11,2
2,1963-05-27,1963,5,27
3,1971-03-03,1971,3,3
4,1961-04-26,1961,4,26


In [86]:
data_df.info()
# 숫자

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이름      300 non-null    object
 1   이미지     300 non-null    object
 2   생년월일    300 non-null    object
 3   정당      300 non-null    object
 4   소속위원회   300 non-null    object
 5   당선횟수    300 non-null    object
 6   사무실전화   300 non-null    object
 7   홈페이지    300 non-null    object
 8   이메일     300 non-null    object
 9   보좌관     300 non-null    object
 10  비서관     300 non-null    object
 11  비서      300 non-null    object
 12  취미, 특기  300 non-null    object
 13  선거구     300 non-null    object
 14  당선횟수2   300 non-null    object
 15  year    300 non-null    int64 
 16  month   300 non-null    int64 
 17  day     300 non-null    int64 
dtypes: int64(3), object(15)
memory usage: 42.3+ KB


In [93]:
#year 최대, year 최소
data_df['year'].max()
data_df['year'].min()

1947

In [97]:
data_df.loc[data_df['year']==1992]
#data_df.loc[data_df['year']==data_df['year'].max(),['이름','정당']]
data_df.loc[data_df['year']==1947]

,이름,이미지,생년월일,정당,소속위원회,당선횟수,사무실전화,홈페이지,이메일,보좌관,비서관,비서,"취미, 특기",선거구,당선횟수2,year,month,day
65,김진표,https://www.assembly.go.kr/photo/9770982.jpg,1947-05-04,더불어민주당,국방위원회,"5선(17대,18대,19대,20대,21대)",02-784-3807,,,"이병진,조기훈","김선회,이현진","김인재,홍승화,김보경,최은정,김가원",,경기수원시무,5선,1947,5,4
290,홍문표,https://www.assembly.go.kr/photo/9770886.jpg,1947-10-05,미래통합당,농림축산식품해양수산위원회,"4선(17대,19대,20대,21대)",02-784-9587,,mphonglove@hanmail.net,"구성완,최재구",신충식,"안진호,손병관,김슬기,구민희,이진선","바둑,독서,산책",충남홍성군예산군,4선,1947,10,5


In [101]:
data_df.loc[data_df['year']==data_df['year'].max(),['이름','정당','year']]
data_df.loc[data_df['year']==data_df['year'].min(),['이름','정당','year']]

,이름,정당,year
65,김진표,더불어민주당,1947
290,홍문표,미래통합당,1947


In [111]:
#나이를 계싼하는 함수 
from datetime import date
def calc_age(dtob):
    my_today = date.today()
    #print(my_today)
    return my_today.year - dtob.year - ((my_today.month, my_today.day)<(dtob.month, dtob.day))


age_list =[]
for idx,row in data_df.iterrows():
    age = calc_age(date(row['year'], row['month'],row['day']))
    #print(idx, row['year'], row['month'],row['day'])
    #print(idx, age)
    age_list.append(age)
    
print(len(age_list), age_list[0:4])    
data_df['나이'] = age_list


300 [60, 60, 57, 49]


In [113]:
data_df.loc[:,['생년월일','나이']].head()

,생년월일,나이
0,1960-06-04,60
1,1959-11-02,60
2,1963-05-27,57
3,1971-03-03,49
4,1961-04-26,59


In [117]:
print(data_df['나이'].max())

73


In [123]:
#선거구2 컬럼을 추가
data_df['선거구'].head()
#우선 str로 만들워줘야
temp_str = data_df['선거구'].str
data_df['선거구2'] = temp_str[0:2]
data_df.loc[:,['선거구','선거구2']].head()

,선거구,선거구2
0,경남창원시성산구,경남
1,대구동구을,대구
2,경기안양시만안구,경기
3,경남진주시을,경남
4,비례대표,비례


In [124]:
data_df['선거구2'].value_counts(normalize=True)

경기    0.196667
서울    0.163333
비례    0.156667
부산    0.060000
경남    0.053333
경북    0.043333
인천    0.043333
대구    0.040000
충남    0.036667
전남    0.033333
전북    0.033333
광주    0.026667
강원    0.026667
충북    0.026667
대전    0.023333
울산    0.020000
제주    0.010000
세종    0.006667
Name: 선거구2, dtype: float64

In [126]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이름      300 non-null    object
 1   이미지     300 non-null    object
 2   생년월일    300 non-null    object
 3   정당      300 non-null    object
 4   소속위원회   300 non-null    object
 5   당선횟수    300 non-null    object
 6   사무실전화   300 non-null    object
 7   홈페이지    300 non-null    object
 8   이메일     300 non-null    object
 9   보좌관     300 non-null    object
 10  비서관     300 non-null    object
 11  비서      300 non-null    object
 12  취미, 특기  300 non-null    object
 13  선거구     300 non-null    object
 14  당선횟수2   300 non-null    object
 15  year    300 non-null    int64 
 16  month   300 non-null    int64 
 17  day     300 non-null    int64 
 18  나이      300 non-null    int64 
 19  선거구2    300 non-null    object
dtypes: int64(4), object(16)
memory usage: 47.0+ KB
